Please install QOKit from source. If you have issues with urllib3, downgrade it after QOKit install by running `pip install urllib3==1.26.6`

First, we check that all the simulators are loaded properly

In [1]:
from qokit.fur import get_available_simulator_names

print(get_available_simulator_names("x"))

['gpu', 'c', 'python']


The output should be `['gpu', 'c', 'python']`. `'c'` simulator is essential to this benchmark. If it's missing, you can compile it manually by running `make -C qokit/fur/c/csim/src/` in the home directory of QOKit

In [2]:
import numpy as np
from tqdm import tqdm
import networkx as nx
import timeit 
from qokit.qaoa_objective_labs import get_qaoa_labs_objective
from qokit.qaoa_objective_maxcut import get_qaoa_maxcut_objective
from qokit.qaoa_objective import get_qaoa_objective

Note that `f_labs` requires a precomputed diagonal for higher `N`. 
You can precompute the diagonal once using 
```
from qokit.labs import negative_merit_factor_from_bitstring
ens = precompute_energies(negative_merit_factor_from_bitstring, N)
outpath = f"../qokit/assets/precomputed_merit_factors/precomputed_energies_{N}.npy"
np.save(outpath, ens, allow_pickle=False)
```
and saving it on disc under `qokit/assets/precomputed_merit_factors`. Note that precomputation can take a while.

In [5]:
# number of qubits
for N in [20, 22, 24]:
    print(f"N={N}")
    # QAOA depth
    p = 6

    theta = np.random.uniform(0,1,2*p)
    G = nx.random_regular_graph(4, N, seed=42)

    # Function initialization may not be fast
    f_maxcut = get_qaoa_maxcut_objective(N, p, G, simulator='c')
    f_labs = get_qaoa_labs_objective(N, p, simulator='c')

    # Function evaluation is fast
    for f, label in [(f_labs, "LABS"), (f_maxcut, "MaxCut")]:
        f(theta) # do not count the first evaluation
        times = []
        for _ in tqdm(range(10)):
            start = timeit.default_timer()
            f(theta)
            end = timeit.default_timer()
            times.append(end-start)
        print(f"\t{label} finished in {np.mean(times):.4f} on average, min: {np.min(times):.4f}, max: {np.max(times):.4f}")

N=20


100%|██████████| 10/10 [00:03<00:00,  3.00it/s]


	LABS finished in 0.3290 on average, min: 0.3222, max: 0.3425


100%|██████████| 10/10 [00:03<00:00,  3.02it/s]


	MaxCut finished in 0.3266 on average, min: 0.3116, max: 0.3582
N=22


...

	MaxCut finished in 1.0919 on average, min: 1.0831, max: 1.1182
N=24


100%|██████████| 10/10 [00:45<00:00,  4.50s/it]


	LABS finished in 4.5025 on average, min: 4.4618, max: 4.5677


100%|██████████| 10/10 [00:43<00:00,  4.38s/it]

	MaxCut finished in 4.3782 on average, min: 4.3449, max: 4.4102
